### vggネットワークの実装

In [7]:
'''
1. vggネットワークを生成する関数の定義 
'''
import torch.nn as nn

def make_vgg():
    '''
    Returns:
        (nn.ModuleList): vggのモジュール（部品）のリスト
    '''
    layers = []
    in_channels = 3

    # vggに配置する畳み込み層のフィルター数（チャネル数に相当）
    # 'M', 'MC'にプーリング層を示す
    cfg = [
        64, 64, 'M',
        128, 128, 'M',
        256, 256, 256, 'MC',
        512, 512, 512, 'M',
        512, 512, 512,
    ]
    # vgg1~vgg5の畳み込み層までを生成
    for v in cfg:
        # vgg1, vgg2, vgg4のプーリング層
        if v == 'M':
            layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
        # vgg3のプーリング層
        elif v == 'MC':
            # vgg3のプーリングで（75、75）の特徴マップを半分のサイズにする際に、
            # ceil_modeをTrueにすることで75/2=37.5を切り上げて38にする
            # この結果vgg3のプーリング層から出力される特徴マップのサイズは
            # (38, 38)になる
            layers += [nn.MaxPool2d(kernel_size=2, stride=2, ceil_mode=True)]
        
        # vgg1~vgg5の畳み込み層
        else:
            conv2d = nn.Conv2d(in_channels, v, kernel_size=3, padding=1)

            # 畳み込み層に活性化関数ReLuをセットしてlayersに追加
            # inplace=TrueにするとReLUへの入力値は保存されない
            layers += [conv2d, nn.ReLU(inplace=True)]
            # チャネル数を出力時のチャネル数（フィルター数）に置き換える
            in_channels = v
        
    # vgg5のプーリング層
    pool5 = nn.MaxPool2d(kernel_size=3, stride=1, padding=1)

    # vgg6の畳み込み層1
    conv6 = nn.Conv2d(512, 1024, kernel_size=3, padding=6, dilation=6)

    # vgg6の畳み込み層2
    conv7 = nn.Conv2d(1024, 1024, kernel_size=1)
    # vgg5のプーリング層、vgg6の畳み込み層1と畳み込み層2をlayeresに追加
    layers += [
        pool5,
        conv6,
        nn.ReLU(inplace=True),
        conv7,
        nn.ReLU(inplace=True),
    ]

    # リストlayersをnn.ModuleListに格納してReturnする
    return nn.ModuleList(layers)

In [8]:
vgg = make_vgg()
print(vgg)

ModuleList(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace=True)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace=True)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU(inplace=True)
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU(inplace=True)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace=True)
  (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): ReLU(inplace=True)
  (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU(inplace=True)
  (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=True)
  (17): Conv2d(256, 512, kernel_siz

### extrasネットワークの実装

In [11]:
'''
2. extrasネットワークを生成する関数の定義
'''
def make_extras():
    '''
    Returns:
        (nn.ModuleList): extrasのモジュール（部品）のリスト
    '''
    layers = []
    in_channels = 1024

    # vggに配置する畳み込み層のフィルター数（チャネル数に相当）
    cfg = [
        256, 512,
        128, 256,
        128, 256,
        128, 256
    ]

    # extras1
    # 出力の形状：(バッチサイズ、512、10、10)
    layers += [nn.Conv2d(in_channels, cfg[0], kernel_size=(1))]
    layers += [nn.Conv2d(cfg[0], cfg[1], kernel_size=(3), stride=2, padding=1)]

    # extras2
    # 出力の形状：(バッチサイズ、256、5、5)
    layers += [nn.Conv2d(cfg[1], cfg[2], kernel_size=(1))]
    layers += [nn.Conv2d(cfg[2], cfg[3], kernel_size=(3), stride=2, padding=1)]

    # extras3
    # 出力の形状：(バッチサイズ、256、3、3)
    layers += [nn.Conv2d(cfg[3], cfg[4], kernel_size=(1))]
    layers += [nn.Conv2d(cfg[4], cfg[5], kernel_size=(3))]

    # extras4
    # 出力の形状：(バッチサイズ、256、1、1)
    layers += [nn.Conv2d(cfg[5], cfg[6], kernel_size=(1))]
    layers += [nn.Conv2d(cfg[6], cfg[7], kernel_size=(3))]

    # リストlayersをnn.ModuleListの格納してReturnする
    return nn.ModuleList(layers)

In [12]:
extras = make_extras()
print(extras)

ModuleList(
  (0): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
  (1): Conv2d(256, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (2): Conv2d(512, 128, kernel_size=(1, 1), stride=(1, 1))
  (3): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (4): Conv2d(256, 128, kernel_size=(1, 1), stride=(1, 1))
  (5): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1))
  (6): Conv2d(256, 128, kernel_size=(1, 1), stride=(1, 1))
  (7): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1))
)


### locネットワークの実装

In [13]:
'''
3. locネットワークを生成する関数の定義
'''
def make_loc(dbox_num=[4, 6, 6, 6, 4, 4]):
    '''
    デフォルトボックスのオフセットを出力するlocネットワークを生成

    Parameters:
        dbox_num(intのリスト):
            out1~out6それぞれに用意されるデフォルトボックスの数
    Returns:
        (nn.ModuleList): locのモジュール（部品）のリスト
    '''
    # ネットワークのモジュールを格納するリスト
    loc_layers = []
    # vgg4の畳み込み層3からの出力にL2Normでの正規化の処理を適用した
    # out1に対する畳み込み層1
    loc_layers += [nn.Conv2d(512, dbox_num[0] * 4, kernel_size=3, padding=1)]

    # vgg6からの最終出力out2に対する畳み込み層2
    loc_layers += [nn.Conv2d(1024, dbox_num[1] * 4, kernel_size=3, padding=1)]

    # extrasのext1からの出力out3に対する畳み込み層3
    loc_layers += [nn.Conv2d(512, dbox_num[2] * 4, kernel_size=3, padding=1)]

    # extrasのext2からの出力out4に対する畳み込み層4
    loc_layers += [nn.Conv2d(256, dbox_num[3] * 4, kernel_size=3, padding=1)]

    # extrasのext3からの出力out5に対する畳み込み層5
    loc_layers += [nn.Conv2d(256, dbox_num[4] * 4, kernel_size=3, padding=1)]

    # extrasのext4からの出力out5に対する畳み込み層6
    loc_layers += [nn.Conv2d(256, dbox_num[5] * 4, kernel_size=3, padding=1)]

    # リストloc_layersをnn.ModuleListに格納してReturnする
    return nn.ModuleList(loc_layers)

In [14]:
loc = make_loc()
print(loc)

ModuleList(
  (0): Conv2d(512, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): Conv2d(1024, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (2): Conv2d(512, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): Conv2d(256, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (4): Conv2d(256, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (5): Conv2d(256, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
)


### confネットワーク

In [15]:
'''
4. confネットワークを生成する関数の定義
'''
def make_conf(classes_num=21, dbox_num=[4, 6, 6, 6, 4, 4]):
    '''
    デフォルトボックスに対する各クラスの確率を出力するネットワークを生成

    Parameters:
        class_num(int): クラスの数
        dbox_num(intのリスト):
            out1~out6それぞれに用意されるデフォルトボックスの数

    Returns:
        (nn.ModuleList): confのモジュール（部品）のリスト
    '''
    # ネットワークのモジュールを格納するリスト
    conf_layers = []

    # vgg4の畳み込み層3からの出力にL2Normでの正規化の処理を適用した
    # out1に対する畳み込み層1
    conf_layers += [nn.Conv2d(512, dbox_num[0] * classes_num, kernel_size=3, padding=1)]

    # vgg6からの最終出力out2に対する畳み込み層2
    conf_layers += [nn.Conv2d(1024, dbox_num[1] * classes_num, kernel_size=3, padding=1)]

    # extrasのext1からの出力out3に対する畳み込み層3
    conf_layers += [nn.Conv2d(512, dbox_num[2] * classes_num, kernel_size=3, padding=1)]

    # extrasのext2からの出力out4に対する畳み込み層4
    conf_layers += [nn.Conv2d(256, dbox_num[3] * classes_num, kernel_size=3, padding=1)]

    # extrasのext3からの出力out5に対する畳み込み層5
    conf_layers += [nn.Conv2d(256, dbox_num[4] * classes_num, kernel_size=3, padding=1)]

    # extrasのext4からの出力out6に対する畳み込み層6
    conf_layers += [nn.Conv2d(256, dbox_num[5] * classes_num, kernel_size=3, padding=1)]

    # リストconf_layersをnn.ModuleListに格納してReturnする
    return nn.ModuleList(conf_layers)

In [16]:
conf = make_conf()
print(conf)

ModuleList(
  (0): Conv2d(512, 84, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): Conv2d(1024, 126, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (2): Conv2d(512, 126, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): Conv2d(256, 126, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (4): Conv2d(256, 84, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (5): Conv2d(256, 84, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
)


### L2Norm層の実装

In [17]:
'''
5. L2ノルムで正規化する層を生成するクラス
'''
import torch
import torch.nn.init as init

class L2Norm(nn.Module):
    '''
    vgg4の畳み込み層3の出力out1をL2ノルムで正規化する層

    Attributes:
        weight: L2Norm層のパラメーター（重み）
        scale: 重みの初期値
        eps: L2ノルムの値に加算する極小値
    '''
    def __init__(self, input_channels=512, scale=20):
        '''
        インスタンス変数の初期化を行う

        Parameters:
            input_channels(int):
                入力データ（vgg4の出力）のチャネル数（デフォルト値512）
            scale(int):
                重みweightの初期値として設定する値（デフォルト値20）
        '''
        super(L2Norm, self).__init__()
        # レイヤーの重みとして（512,）の1階テンソルを配置
        self.weight = nn.Parameter(torch.Tensor(input_channels))
        self.scale = scale
        self.reset_parameters()
        self.eps = 1e-10
    
    def reset_parameters(self):
        '''
        全ての重みをscaleの値で初期化する
        '''
        # torch.nn.init_constant_()で重みテンソルに初期値を設定
        # weightの値を全てscale(=20)にする
        init.constant_(self.weight, self.scale)
    
    def forward(self, x):
        '''
        L2Normにおける順伝播を行う

        Parameters:
            x(Tensor):
                vgg4の畳み込み層3からの出力(バッチサイズ、512、38、38)
        
        Returns:
            L2ノルムで正規化した後、scale(=20)の重みを適用した
            (バッチサイズ、512、38、38)の4階テンソル
        '''
        # 各チャネルにおける38*38個の特徴量の個々のセルについて、
        # チャネル方向の2乗和の平方根(L2ノルム)を求め、極小値epsを加算する
        # normの形状は（バッチサイズ、1、38、38）
        norm = x.pow(2).sum(dim=1, keepdim=True).sqrt() + self.eps
        # 各チャネルにおける38*38個の特徴量の個々のセルの値を
        # 同じセルのnormで割って正規化する
        x = torch.div(x, norm)

        # self.weightの1階テンソル（512、）を（バッチサイズ、512、38、38）の
        # 4階テンソルに変形してxと同じ形状にする
        weights = self.weight.unsqueeze(0).unsqueeze(3).expand_as(x)
        # 変形後のxに重みを適用する
        out = weights * x

        return out

In [18]:
import itertools
import pprint

l1 = ['a', 'b', 'c']
l2 = ['1', '2', '3']
p = itertools.product(l1, l2)
for v in p:
    print(v)

('a', '1')
('a', '2')
('a', '3')
('b', '1')
('b', '2')
('b', '3')
('c', '1')
('c', '2')
('c', '3')


In [19]:
'''
6. デフォルトボックスを出力するクラス
'''
from itertools import product as product
from math import sqrt as sqrt

class DBox(object):
    '''
    8732個のDBoxの(x座標、y座標、幅、高さ)を生成する

    Attributes:
        image_size(int): イメージのサイズ
        feature_maps(list): out1~out6における特徴量マップのサイズのリストを保持
        num_priors(int): featire_mapsの要素数、out1~out6の個数6を保持
        steps(list): DBoxのサイズのリストを保持
        min_sizes(list): 小さい正方形のDBoxのサイズを保持
        max_sizes(list): 大きい正方形のDBoxのサイズを保持
        aspect_rations(list): 長方形のDBoxのアスペクト比を保持
    '''
    def __init__(self, cfg):
        '''
        インスタンス変数の初期化を行う
        '''
        super(DBox, self).__init__()

        # 画像サイズ（300）を設定
        self.image_size = cfg['input_size']
        # out1~out6における特徴量マップのサイズ[38, 19,...]を設定
        self.feature_maps = cfg['feature_maps']
        # out1~out6の個数＝6を設定
        self.num_priors = len(cfg['feature_maps'])
        # DBoxのサイズ[8, 16, 32, ...]を設定
        self.steps = cfg['steps']
        # 小さい正方形のDBoxのサイズ[30, 60, 111, ...]を設定
        self.min_sizes = cfg['min_sizes']
        # 大きい正方形のDBoxのサイズ[60, 111, 162, ...]を設定
        self.max_sizes = cfg['max_sizes']
        # 長方形のDBoxのアスペクト比[[2], [2, 3], [2, 3], ...]を設定
        self.aspect_rations = cfg['aspect_rations']
    
    def make_dbox_list(self):
        '''
        DBoxを作成する

        Returns:
            (Tensor)DBoxの[cx, cy, width, weight]を格納した(8732, 4)の形状のテンソル
        '''
        mean = []
        # out1~out6における特徴量マップの数（6）だけ繰り返す
        # 特徴量マップのサイズのリストからインデックスをk, サイズをfに取り出す
        # feature_maps: [38, 19, 10, 5, 3, 1]
        # k: 0, 1,2, 3, 4, 5
        # f: 38, 19, 10, 5, 3, 1
        for k, f in enumerate(self.feature_maps):
            # fまでの数をrepeat=3を指定して2つのリストにして組み合わせ（直積）を作る
            # f=38の場合
            # i: 0, 0, 0, 0,... の38個の0に対して
            # j: 0, 1, 2, 3,... の37を組み合わせる
            # (i, j)は(0, 0)(0, 1)...(0, 37)~(37, 0)...(37, 37)
            for i, j in product(range(f), repeat=2):
                # 特徴量の画像サイズをDBoxのサイズsteps[k]でわる
                # 300 / steps: [8, 16, 32, 64, 100, 300]
                f_k = self.image_size / self.steps[k]

                # 特徴量ごとのDBoxの中心のx座標、y座標を求める
                # (0~1の範囲に規格化)
                cx = (j + 0.5) / f_k
                cy = (i + 0.5) / f_k

                # 小さい正方形のDBoxのサイズmin_sizes[k]を
                # 画像のサイズでわる
                # min_sizes: [30, 50, 111, 162, 213, 264] / 300
                s_k = self.min_sizes[k] / self.image_size
                # 小さい正方形のDBoxの[cx, cy, width, height]をリストに追加
                mean += [cx, cy, s_k, s_k]                

                # 大きい正方形のDBoxのサイズmax_sizes[k]を
                # 画像のサイズでわる
                # max_sizes: [45, 99, 153, 207, 261, 315] / 300
                # さらにs_kをかけて平方根を求める
                s_k_prime = sqrt(s_k * (self.max_sizes[k] / self.image_size))
                # 大きい正方形のDBoxの[cx, cy, width, height]をリストに追加
                mean += [cx, cy, s_k_prime, s_k_prime]
                
                # 長方形のDBoxの[cx, cy, width, height]をリストに追加
                for ar in self.aspect_rations[k]:
                    # widthはs_kにアスペクト比の平方根をかけたもの
                    # heightはs_kにアスペクト比の平方根を割ったもの
                    mean += [cx, cy, s_k * sqrt(ar), s_k / sqrt(ar)]
                    # widthはs_kにアスペクト比と平方根で割ったもの
                    # heightはs_kにアスペクト比と平方根をかけたもの
                    mean += [cx, cy, s_k / sqrt(ar), s_k * sqrt(ar)]
        
        # DBoxの[cx, cy, width, height]のリストを(8732, 4)の2階テンソルに変換
        output = torch.Tensor(mean).view(-1, 4)
        # DBoxの大きさが1を超えている場合は1にする
        output.clamp_(max=1, min=0)

        # DBoxの[cx, cy, width, height]を格納した2階テンソルを返す
        return output

In [20]:
# DBoxクラスの動作確認
import pandas as pd

# デフォルトボックスの生成に必要なパラメーター
dbox_cfg = {
    # 画像の入力サイズ
    'input_size': 300,
    # out1~out6における特徴量マップのサイズ
    'feature_maps': [38, 19, 10, 5, 3, 1],
    # DBoxのサイズ
    'steps': [8, 16, 32, 64, 100, 300],
    # 小さい正方形のDBoxのサイズ
    'min_sizes': [30, 60, 111, 162, 213, 264],
    # 大きい正方形のDBoxのサイズ
    'max_sizes': [60, 111, 162, 213, 264, 315],
    # 長方形のDBoxのアスペクトひ
    'aspect_rations': [[2], [2, 3], [2, 3], [2, 3], [2], [2]]
}

# DBoxオブジェクトを生成
dbox = DBox(dbox_cfg)
# DBoxのリストを取得
dbox_list = dbox.make_dbox_list()

# 出力
print(dbox_list)

tensor([[0.0133, 0.0133, 0.1000, 0.1000],
        [0.0133, 0.0133, 0.1414, 0.1414],
        [0.0133, 0.0133, 0.1414, 0.0707],
        ...,
        [0.5000, 0.5000, 0.9612, 0.9612],
        [0.5000, 0.5000, 1.0000, 0.6223],
        [0.5000, 0.5000, 0.6223, 1.0000]])
